# Seq2Seq Modeling
- source: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

In [2]:
with open("eng-fra.txt", "r", encoding="utf-8") as f:
    data = f.read()
print(f"Number of sentence pairs: {len(data):_}")

Number of sentence pairs: 9_395_786


In [3]:
device = "cuda" if torch.cuda.is_available() else "mps"

## Load data
- Each word represented by a one-hot vector
- Trim the vocab to only use a few thousand words per language
- `Lang`: word-to-index and index-to-word dicts

The full process for preparing the data is:
- Read text file and split into lines, split lines into pairs
- Normalize text, filter by length and content
- Make word lists from sentences in pairs

In [4]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # SOS, EOS

    def addSentence(self, sentence):
        for word in sentence.split(" "):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [5]:
def unicodeToAscii(s):
    return "".join(
        c for c in unicodedata.normalize("NFD", s)
        if unicodedata.category(c) != "Mn"
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

In [9]:
def readLangs(lang1, lang2, reverse=False):
    lines = open(f"{lang1}-{lang2}.txt", encoding="utf-8").read().strip().split("\n")
    pairs = [[normalizeString(s) for s in l.split("\t")] for l in lines]

    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [10]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(" ")) < MAX_LENGTH and \
        len(p[1].split(" ")) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [11]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print(f"Read {len(pairs)} pairs")
    pairs = filterPairs(pairs)
    print(f"Trimmed to {len(pairs)} pairs")

    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])

    print("Counting words..")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)

    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData("eng", "fra", True)
print(random.choice(pairs))

Read 135842 pairs
Trimmed to 11445 pairs
Counting words..
fra 4601
eng 2991
['nous allons toutes a la maison', 'we re all going home']


## Encoder Model
RNN: Creates a single vector out of the entire input sequence which, in the ideal case, encodes the 'meaning' of the input sequence into a single embedding vector. 
- Initial hidden state is zero if not explicitly provided. 

In [12]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input_token):
        embedded = self.embedding(input_token)  # create token embedding
        embedded = self.dropout(embedded)  # apply dropout: randomly zeros some of the embedding vector's elements
        output, hidden = self.gru(embedded)  # rnn: output is a collection of hidden at every time step
        return output, hidden  # hidden is the final hidden vector capturing the meaning of the sequence

## Decoder Model
RNN: Takes the encoder output vector(s) and outputs a sequence of words to create the translation. 

### Simple Decoder
- Only use the last hidden state of the encoder, called the context vector as it encodes the context from the entire sequence.
- Context vector used as the initial hidden state of the decoder.
- Initial input token is the `<SOS>` token. 

In [13]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                decoder_input = target_tensor[:, i].unsqueeze(1)  # 'teacher forcing' from translated ground truth text (training)
            else:
                _, topi = decoder_output.topk(1)  # use its own predictions as the next input (inference)
                decoder_input = topi.squeeze(-1).detach()

        return decoder_outputs, decoder_hidden, None

    def forward_step(self, input_token, hidden):
        embedded = self.embedding(input_token)
        embedded = F.relu(embedded)
        output, hidden = self.gru(embedded, hidden)
        output = self.out(output)
        return output, hidden

### Attention Mechanisms

In [14]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

In [15]:
class LuongAttention(nn.Module):
    def __init__(self, hidden_size, attention_type="general"):
        super(LuongAttention, self).__init__()
        self.attention_type = attention_type

        if self.attention_type == "general":
            self.Wa = nn.Linear(hidden_size, hidden_size, bias=False)

    def forward(self, query, keys):
        if self.attention_type == "dot":
            scores = torch.bmm(query, keys.transpose(1, 2))
        elif self.attention_type == "general":
            scores = torch.bmm(self.Wa(query), keys.transpose(1, 2))

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

### Attention Decoder
- Unlike simple decoder which only uses the last hidden state at each time step, attention decoder uses all hidden states to focus on different parts of the encoder's output.
- Attention weights are multiplied by the encoder output vectors to create a weighted combination. 

In [16]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(decoder_input, decoder_hidden, encoder_outputs)
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                decoder_input = target_tensor[:, i].unsqueeze(1)  # 'teacher forcing' from translated ground truth text (training)
            else:
                _, topi = decoder_output.topk(1)  # use its own predictions as the next input (inference)
                decoder_input = topi.squeeze(-1).detach()

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions

    def forward_step(self, input_token, hidden, encoder_outputs):
        embedded = self.dropout(self.embedding(input_token))
        context, attn_weights = self.attention(
            query=hidden.permute(1, 0, 2), 
            keys=encoder_outputs
        )
        output, hidden = self.gru(torch.cat((embedded, context), dim=2), hidden)
        output = self.out(output)
        return output, hidden, attn_weights

## Training

In [17]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(" ")]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

In [18]:
def get_dataloader(batch_size):
    input_lang, output_lang, pairs = prepareData("eng", "fra", True)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)

        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)

        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(
        torch.LongTensor(input_ids).to(device), 
        torch.LongTensor(target_ids).to(device)
    )
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    return input_lang, output_lang, train_dataloader

In [19]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    total_loss = 0

    for data in dataloader:
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)), 
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [20]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [21]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001, print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0
    plot_loss_total = 0

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()  # negative log-likelihood loss

    for epoch in range(1, n_epochs+1):
        loss = train_epoch(
            train_dataloader, 
            encoder, 
            decoder, 
            encoder_optimizer, 
            decoder_optimizer, 
            criterion
        )
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print(f"{timeSince(start, epoch / n_epochs)} ({epoch} {epoch / n_epochs * 100}%) {print_loss_avg:.4f}")

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

In [22]:
hidden_size = 128
batch_size = 32

input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

train(train_dataloader, encoder, decoder, n_epochs=80, print_every=5, plot_every=5)

Read 135842 pairs
Trimmed to 11445 pairs
Counting words..
fra 4601
eng 2991
1m 13s (- 18m 29s) (5 6.25%) 1.5446
2m 26s (- 17m 5s) (10 12.5%) 0.6897
3m 37s (- 15m 41s) (15 18.75%) 0.3568
4m 49s (- 14m 27s) (20 25.0%) 0.1959
5m 58s (- 13m 7s) (25 31.25%) 0.1203
7m 9s (- 11m 56s) (30 37.5%) 0.0825
8m 22s (- 10m 46s) (35 43.75%) 0.0628
9m 35s (- 9m 35s) (40 50.0%) 0.0520
10m 45s (- 8m 21s) (45 56.25%) 0.0449
11m 53s (- 7m 8s) (50 62.5%) 0.0390
13m 2s (- 5m 55s) (55 68.75%) 0.0370
14m 9s (- 4m 43s) (60 75.0%) 0.0341
15m 18s (- 3m 31s) (65 81.25%) 0.0326
16m 27s (- 2m 21s) (70 87.5%) 0.0312
17m 35s (- 1m 10s) (75 93.75%) 0.0296
18m 44s (- 0m 0s) (80 100.0%) 0.0291
